In [1]:
import datetime as dt
import pandas as pd
import numpy as np

In [2]:
educacion = pd.read_csv('/home/luupesado/7506_Datos/2018/datos_tp/joins/educacion.csv')
genero_y_edad = pd.read_csv('/home/luupesado/7506_Datos/2018/datos_tp/joins/genero_y_edad.csv')

Limpiamos el set de datos de educación para quedarnos con el mayor nivel educativo de cada postulante, tal cual lo hicimos en el tp 1

In [3]:
educacion['nombre'] = educacion['nombre'].astype('string')

In [4]:
import unicodedata
def elimina_tildes(s):
   return ''.join((c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn'))

In [5]:
educacion['nombre'] = educacion['nombre'].apply(lambda x: elimina_tildes(x.decode('utf-8')))

In [6]:
educacion['nombre-estado'] = educacion['nombre'] + ' - ' + educacion['estado']

In [7]:
educacion['nombre-estado'] = educacion['nombre-estado'].astype('category')

In [8]:
categories_order = ['Otro - Abandonado',
 'Otro - En Curso',                                        
 'Otro - Graduado', 
'Secundario - Abandonado',
 'Secundario - En Curso',
 'Secundario - Graduado',
 'Terciario/Tecnico - Abandonado',
 'Terciario/Tecnico - En Curso',
 'Terciario/Tecnico - Graduado',
 'Universitario - Abandonado',
 'Universitario - En Curso',
 'Universitario - Graduado',
'Posgrado - Abandonado',
 'Posgrado - En Curso',
 'Posgrado - Graduado',
 'Master - Abandonado',
 'Master - En Curso',
 'Master - Graduado',
'Doctorado - Abandonado',
 'Doctorado - En Curso',
 'Doctorado - Graduado']

In [9]:
educacion['nombre-estado'] = educacion['nombre-estado'].cat \
                                        .set_categories(categories_order, ordered = True)

In [10]:
educacion = educacion.sort_values('nombre-estado', ascending= False).drop_duplicates(subset='idpostulante', keep = 'first')

Ahora limpiamos el set de datos de genero y edad, tal cual lo hicims el el tp1

In [11]:
import datetime as dt

In [12]:
def strdate_to_age(string):
    
    date = dt.datetime.strptime(string, "%Y-%m-%d")
    today = dt.datetime(2018, 4, 1)
    edad = today.year - date.year
    if today.month >= date.month and today.day >= date.day:
        edad = edad +1
    return edad


In [13]:
genero_y_edad = genero_y_edad.dropna()

In [14]:
genero_y_edad['edad'] = genero_y_edad['fechanacimiento'].apply(lambda x: strdate_to_age(str(x)))

In [15]:
genero_y_edad.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 745747 entries, 0 to 780018
Data columns (total 5 columns):
Unnamed: 0         745747 non-null int64
idpostulante       745747 non-null object
fechanacimiento    745747 non-null object
sexo               745747 non-null object
edad               745747 non-null int64
dtypes: int64(2), object(3)
memory usage: 34.1+ MB


Ahora hago el merge

In [16]:
educacion.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 447909 entries, 1070867 to 641978
Data columns (total 5 columns):
Unnamed: 0       447909 non-null int64
idpostulante     447909 non-null object
nombre           447909 non-null object
estado           447909 non-null object
nombre-estado    447909 non-null category
dtypes: category(1), int64(1), object(3)
memory usage: 17.5+ MB


In [17]:
genero_y_edad.tail()

,Unnamed: 0,idpostulante,fechanacimiento,sexo,edad
780010,297737,PmGx0vJ,1997-02-27,FEM,21
780012,297739,3NPqmBx,1990-01-08,FEM,28
780014,297741,wVkvPrQ,1994-02-24,MASC,24
780015,297742,MVrow99,1999-09-27,FEM,19
780018,297745,X95z68O,1998-10-29,MASC,20


In [18]:
del genero_y_edad['Unnamed: 0']
del educacion['Unnamed: 0']

In [19]:
postulantes = pd.merge(genero_y_edad, educacion, on='idpostulante', how='inner')

In [20]:
postulantes.head()

,idpostulante,fechanacimiento,sexo,edad,nombre,estado,nombre-estado
0,YZP,1961-05-01,MASC,57,Master,En Curso,Master - En Curso
1,ebE,1952-07-07,MASC,66,Posgrado,Graduado,Posgrado - Graduado
2,ebE,1952-07-07,MASC,66,Posgrado,Graduado,Posgrado - Graduado
3,ZjlZ,1970-01-25,MASC,48,Master,Graduado,Master - Graduado
4,ZjlZ,1970-01-25,MASC,48,Master,Graduado,Master - Graduado


In [21]:
postulantes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 693704 entries, 0 to 693703
Data columns (total 7 columns):
idpostulante       693704 non-null object
fechanacimiento    693704 non-null object
sexo               693704 non-null object
edad               693704 non-null int64
nombre             693704 non-null object
estado             693704 non-null object
nombre-estado      693704 non-null category
dtypes: category(1), int64(1), object(5)
memory usage: 37.7+ MB


Ahora filtramos los postulantes por edad, que no habíamos olvidade de hacer antes del merge. NOs quedamos con los postulantes entre 16 y 70 años. Según vimos en el primer TP había muy pocos datos de gente de otras edades. Además los menores de 16 no pueden trabajar legalmente y si bien la edad jubilatoria no es 70 perdemos muy pocos datos con esta restricvción según vimos en el TP1.

In [22]:
postulantes = postulantes[postulantes['edad'] >=16]
postulantes = postulantes[postulantes['edad'] <=70]

In [23]:
postulantes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 693517 entries, 0 to 693703
Data columns (total 7 columns):
idpostulante       693517 non-null object
fechanacimiento    693517 non-null object
sexo               693517 non-null object
edad               693517 non-null int64
nombre             693517 non-null object
estado             693517 non-null object
nombre-estado      693517 non-null category
dtypes: category(1), int64(1), object(5)
memory usage: 37.7+ MB


In [24]:
postulantes['idpostulante'] = postulantes.drop_duplicates(subset = 'idpostulante')

In [25]:
postulantes = postulantes.dropna()

In [26]:
postulantes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 438597 entries, 0 to 693703
Data columns (total 7 columns):
idpostulante       438597 non-null object
fechanacimiento    438597 non-null object
sexo               438597 non-null object
edad               438597 non-null int64
nombre             438597 non-null object
estado             438597 non-null object
nombre-estado      438597 non-null category
dtypes: category(1), int64(1), object(5)
memory usage: 23.8+ MB


In [27]:
postulantes.to_csv('/home/luupesado/7506_Datos/2018/TPsDatos_1C2018/TP2/Archivos_a_usar/postulantes')